In [53]:
import numpy as np
import os
import pandas as pd
import pickle
import sklearn

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [52]:
year = 2023
month = 3
taxi_type = "yellow"
input_file = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet"
output_file = f"processed_{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet"
model_path = "model.bin"

CATEGORICAL = ['PULocationID', 'DOLocationID']
TARGET = 'duration'

In [71]:
def read_data(file_name):
    df = pd.read_parquet(file_name)
    df[TARGET] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df[TARGET] = df[TARGET].dt.total_seconds() / 60
    df = df[(df[TARGET] >= 1) & (df[TARGET] <= 60)].copy()
    df["ride_id"] = f'{year:04d}/{month:02d}_' + test_data.index.astype('str')
    return df

def prepare_data(df):
    df[CATEGORICAL] = df[CATEGORICAL].fillna(-1).astype('int').astype('str')
    X = df[CATEGORICAL].to_dict(orient='records')
    target = data[TARGET].values

    return X, target

def load_model():
    with open(model_path, "rb") as f:
        dv, lr = pickle.load(f)

    model = Pipeline([
        ('vectorizer', dv),
        ('regression', lr)
    ])

    return model

def apply_model(file_name, output_file):
    model = load_model()
    data = read_data(file_name)
    X, target = prepare_data(data.copy())
    predictions = model.predict(X)

    result = pd.DataFrame()
    result["ride_id"] = data["ride_id"]
    result[TARGET] = predictions

    result.to_parquet(
        output_file,
        engine='pyarrow'
    )

In [15]:
result = lr.predict(x_test)
np.std(result)

6.247483064778619

In [72]:
apply_model(input_file, output_file)

In [73]:
! du -sh ./processed_yellow_tripdata_2023-03.parquet

 36M	./processed_yellow_tripdata_2023-03.parquet


In [76]:
predictions.mean()

14.203865642696083